In [1]:
!pip install obspy
!pip install numpy pandas matplotlib
!pip install scipy
!pip install keras
!pip install tensorflow
!pip install wurlitzer

import os
import obspy.clients.fdsn
import numpy as np
import pandas as pd
import obspy
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
from datetime import datetime
import scipy
import warnings
warnings.filterwarnings("ignore")


import pylab as plt
import warnings
import os
import h5py
import matplotlib as mpl
%matplotlib inline

import sklearn
from sklearn import linear_model

import pandas as pd
import seaborn as sns

import keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, LSTM
from keras import losses
from keras.utils import to_categorical
from keras.layers import BatchNormalization
import keras.backend as K

from __future__ import print_function

from collections import defaultdict
import pickle
from PIL import Image

from six.moves import range

from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Embedding, Dropout, LeakyReLU
from keras.layers import Conv2D, MaxPooling2D
from keras.models import Model
from keras.optimizers import Adam
from keras.utils import Progbar
import numpy as np

from numpy import zeros
from numpy import ones
from numpy import expand_dims
from numpy.random import randn
from numpy.random import randint
from keras.optimizers import Adam
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import BatchNormalization
from keras.layers import Dropout
from keras.layers import Embedding
from keras.layers import Activation
from keras.layers import Concatenate
from keras.initializers import RandomNormal
from matplotlib import pyplot

np.random.seed(42)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.5/14.5 MB 61.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 42.8 MB/s eta 0:00:00:00:01
^C
ERROR: Operation cancelled by user
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 25.5 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: keras
    Found existing installation: keras 3.4.1
    Uninstalling keras-3.4.1:
      Successfully uninstalled keras-3.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.


ModuleNotFoundError: No module named 'obspy'

In [ ]:
import os
os.getcwd()

In [ ]:
import numpy as np
import os

# Define the folder path
folder_path = '/kaggle/input/dataset-noise'

# Load numpy arrays from the specified folder
W = np.load(os.path.join(folder_path, '/kaggle/input/dataset-noise/IRIS_dataset_waveforms_train(1).npy'))
Y = np.load(os.path.join(folder_path, '/kaggle/input/dataset-noise/IRIS_dataset_labels_train(1).npy'))

print(W.shape, Y.shape)

In [ ]:
W = W.reshape(-1,12000,3,1)
W.shape

In [ ]:
import numpy as np

def add_multi_type_noise(clean_data):
    noisy_data = clean_data.copy()
    noise_types = []
    
    # Gaussian noise
    if np.random.random() < 0.5:
        noise = np.random.normal(0, 0.1, noisy_data.shape)
        noisy_data += noise
        noise_types.append('gaussian')
    
    # Salt and Pepper noise
    if np.random.random() < 0.5:
        salt = np.random.random(noisy_data.shape) < 0.05
        pepper = np.random.random(noisy_data.shape) < 0.05
        noisy_data[salt] = 1
        noisy_data[pepper] = -1
        noise_types.append('salt_and_pepper')
    
    # Speckle noise
    if np.random.random() < 0.5:
        noise = np.random.randn(*noisy_data.shape)
        noisy_data += noisy_data * noise * 0.1
        noise_types.append('speckle')
    
    return noisy_data, noise_types

# Update W and Y
W_noisy = np.array([add_multi_type_noise(w)[0] for w in W])

# Update Y to be the clean data
Y = W.copy()

# Reshape W_noisy and Y if necessary
W_noisy = W_noisy.reshape(-1, 3, 12000, 1)
Y = Y.reshape(-1, 3, 12000, 1)

# Convert to float32
W_noisy = W_noisy.astype('float32')
Y = Y.astype('float32')

print("W_noisy shape:", W_noisy.shape)
print("Y shape:", Y.shape)

# Save W_noisy
np.save('/kaggle/working/W_noisy.npy', W_noisy)

# Save Y
np.save('/kaggle/working/Y_clean.npy', Y)

print("Data saved successfully using numpy.")

In [ ]:
from matplotlib.pyplot import plot
for i in range(3):
    plt.plot(W_noisy[23, i, :])

In [ ]:
from matplotlib.pyplot import plot
for i in range(3):
    plt.plot(Y[232, i, :])

In [ ]:
from sklearn.model_selection import train_test_split
import tensorflow as tf

# First, split into train+val and test sets
W_trainval, W_test, Y_trainval, Y_test = train_test_split(W_noisy, Y, test_size=0.2, random_state=42)

# Then split train+val into train and validation sets
W_train, W_val, Y_train, Y_val = train_test_split(W_trainval, Y_trainval, test_size=0.2, random_state=42)

print("Train set shape:", W_train.shape)
print("Validation set shape:", W_val.shape)
print("Test set shape:", W_test.shape)

# Create TensorFlow datasets
batch_size = 32  # Adjust this based on your GPU memory

train_dataset = tf.data.Dataset.from_tensor_slices((W_train, Y_train))
train_dataset = train_dataset.shuffle(1000).batch(batch_size).prefetch(tf.data.AUTOTUNE)

val_dataset = tf.data.Dataset.from_tensor_slices((W_val, Y_val))
val_dataset = val_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)

test_dataset = tf.data.Dataset.from_tensor_slices((W_test, Y_test))
test_dataset = test_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)

# Additional preprocessing steps
def preprocess(noisy, clean):
    # Ensure data types are correct
    noisy = tf.cast(noisy, tf.float32)
    clean = tf.cast(clean, tf.float32)
    
    # Data augmentation (if needed)
    # For example, you could add random flipping:
    if tf.random.uniform(()) > 0.5:
        noisy = tf.reverse(noisy, axis=[1])
        clean = tf.reverse(clean, axis=[1])
    
    return noisy, clean

# Apply preprocessing to the datasets
train_dataset = train_dataset.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
val_dataset = val_dataset.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
test_dataset = test_dataset.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)

# Check the shapes
for noisy_batch, clean_batch in train_dataset.take(1):
    print("Batch shapes:")
    print("Noisy:", noisy_batch.shape)
    print("Clean:", clean_batch.shape)

# Calculate the number of batches
train_batches = tf.data.experimental.cardinality(train_dataset).numpy()
val_batches = tf.data.experimental.cardinality(val_dataset).numpy()
test_batches = tf.data.experimental.cardinality(test_dataset).numpy()

print(f"Number of training batches: {train_batches}")
print(f"Number of validation batches: {val_batches}")
print(f"Number of test batches: {test_batches}")

In [ ]:
from matplotlib.pyplot import plot
for i in range(3):
    plt.plot(W_test[13, i, :])

In [ ]:
from matplotlib.pyplot import plot
for i in range(3):
    plt.plot(Y_test[13, i, :])

In [ ]:
def remove_nan_inf(noisy, clean):
    noisy = tf.where(tf.math.is_finite(noisy), noisy, 0.0)
    clean = tf.where(tf.math.is_finite(clean), clean, 0.0)
    return noisy, clean

train_dataset = train_dataset.map(remove_nan_inf, num_parallel_calls=tf.data.AUTOTUNE)
val_dataset = val_dataset.map(remove_nan_inf, num_parallel_calls=tf.data.AUTOTUNE)
test_dataset = test_dataset.map(remove_nan_inf, num_parallel_calls=tf.data.AUTOTUNE)

In [ ]:
'''print("Train label distribution:", np.bincount(Y_train.flatten().astype(int)))
print("Validation label distribution:", np.bincount(Y_val.flatten().astype(int)))
print("Test label distribution:", np.bincount(Y_test.flatten().astype(int)))
'''

In [ ]:
'''
from tensorflow.keras import mixed_precision

policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)'''

In [ ]:
import numpy as np
import tensorflow as tf
tf.keras.mixed_precision.set_global_policy('mixed_float16')
from tensorflow.keras.layers import Input, Dense, Reshape, Concatenate, Conv2D, BatchNormalization, Activation, Embedding, Flatten, UpSampling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from collections import defaultdict
from tensorflow.keras.utils import Progbar
from tensorflow.keras.layers import MaxPooling2D

from tensorflow.keras import layers, models, Input

import tensorflow as tf
from tensorflow.keras import layers

def build_generator(latent_dim=100):
    # Noisy waveform input
    waveform_input = tf.keras.layers.Input(shape=(3, 12000, 1), name='waveform_input')
    x_waveform = tf.keras.layers.Lambda(lambda x: tf.cast(x, tf.float32))(waveform_input)
    
    # Noise input
    noise_input = tf.keras.layers.Input(shape=(latent_dim,), name='noise_input')
    
    # Process noise input
    x_noise = layers.Dense(3 * 12000 * 1)(noise_input)
    x_noise = layers.Reshape((3, 12000, 1))(x_noise)
    
    # Add noise to waveform input instead of concatenating
    x = layers.Add()([x_waveform, x_noise])
    
    # Encoder
    encoder_layers = []
    for filters in [64, 128, 256, 512]:
        x = layers.Conv2D(filters, (3, 3), strides=(1, 2), padding='same')(x)
        x = layers.BatchNormalization()(x)
        x = layers.LeakyReLU(alpha=0.2)(x)
        encoder_layers.append(x)
    
    # Bottleneck
    x = layers.Conv2D(512, (3, 3), strides=(1, 2), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU(alpha=0.2)(x)
    
    # Decoder
    for filters in [512, 256, 128, 64]:
        x = layers.Conv2DTranspose(filters, (3, 3), strides=(1, 2), padding='same')(x)
        x = layers.BatchNormalization()(x)
        x = layers.LeakyReLU(alpha=0.2)(x)
        if len(encoder_layers) > 0:  # Skip connection
            skip_connection = encoder_layers.pop()
            x = layers.Concatenate()([x, skip_connection])
    
    # Output layer
    x = layers.Conv2DTranspose(filters, (3, 3), strides=(1, 2), padding='same')(x)
    x = layers.Conv2DTranspose(1, (3, 3), strides=(1, 1), padding='same', activation='tanh')(x)
    
    x = tf.cast(x, tf.float32)
    return tf.keras.Model(inputs=[waveform_input, noise_input], outputs=x)

generator = build_generator()
generator.summary()


In [ ]:
pip install tensorflow-addons

In [ ]:
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.constraints import max_norm
from tensorflow_addons.layers import SpectralNormalization

class SpectralNormalizationConstraint(tf.keras.constraints.Constraint):
    def __init__(self):
        self.iteration = 0

    def __call__(self, w):
        self.iteration += 1
        if self.iteration % 50 == 0:  # Update u every 50 iterations
            self.w_shape = w.shape
            w = tf.reshape(w, [-1, w.shape[-1]])
            u = tf.random.normal([w.shape[0], 1])
            v = tf.random.normal([w.shape[1], 1])
            for _ in range(10):  # Power iteration
                v = tf.matmul(tf.transpose(w), u)
                v = v / tf.norm(v)
                u = tf.matmul(w, v)
                u = u / tf.norm(u)
            sigma = tf.matmul(tf.matmul(tf.transpose(u), w), v)
            w = w / sigma
            w = tf.reshape(w, self.w_shape)
        return w

def build_discriminator():
    inputs = tf.keras.layers.Input(shape=(3, 12000, 1))
    x = tf.keras.layers.Lambda(lambda x: tf.cast(x, tf.float32))(inputs)
    
    for filters in [64, 128, 256, 512]:
        x = layers.Conv2D(filters, (3, 3), strides=(1, 2), padding='same', 
                          kernel_constraint=SpectralNormalizationConstraint())(x)
        x = layers.BatchNormalization()(x)
        x = layers.LeakyReLU(alpha=0.2)(x)
    
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(1, activation='sigmoid', 
                     kernel_constraint=SpectralNormalizationConstraint())(x)
    
    x = tf.cast(x, tf.float32)
    return tf.keras.Model(inputs=inputs, outputs=x)

discriminator = build_discriminator()
discriminator.summary()


In [ ]:
def calculate_psnr(clean, denoised):
    mse = np.mean((clean - denoised) ** 2)
    if mse == 0:
        return 100
    max_pixel = 1.0
    psnr = 20 * np.log10(max_pixel / np.sqrt(mse))
    pnsr = tf.cast(pnsr, tf.float32)
    return psnr

def calculate_ssim(clean, denoised):
    ssim = structural_similarity(clean, denoised, data_range=1.0)
    ssim = tf.cast(ssim, tf.float32)
    return ssim

In [ ]:
'''
import tensorflow as tf
tf.keras.mixed_precision.set_global_policy('mixed_float16')
import numpy as np
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from sklearn.model_selection import train_test_split
from skimage.metrics import structural_similarity
import logging

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Hyperparameters
nb_epochs = 20
batch_size = 8
latent_size = 100
adam_lr = 0.0002
adam_beta_1 = 0.5
lambda_l1 = 100  # Weight for L1 loss

# Load and preprocess data
W = np.load('/kaggle/input/dataset-noise/IRIS_dataset_waveforms_train(1).npy')
Y = np.load('/kaggle/input/dataset-noise/IRIS_dataset_labels_train(1).npy')

# Reshape W to (samples, 3, 12000, 1)
W = W.reshape(-1, 3, 12000, 1)

# Convert Y to the same shape as W for clean waveforms
Y = Y.reshape(-1, 1, 1, 1)
Y = np.tile(Y, (1, 3, 12000, 1))

# Convert to float32
W = W.astype('float32')
Y = Y.astype('float32')

# Normalize the data to [-1, 1] range
W = (W - W.min()) / (W.max() - W.min()) * 2 - 1
Y = (Y - Y.min()) / (Y.max() - Y.min()) * 2 - 1

# Split data
W_train, W_test, Y_train, Y_test = train_test_split(W, Y, test_size=0.2, random_state=42)
W_train, W_val, Y_train, Y_val = train_test_split(W_train, Y_train, test_size=0.2, random_state=42)

# Build models (assuming you've defined build_generator and build_discriminator functions)

generator = build_generator(latent_size)
test_noise = tf.random.normal([1, latent_size])
test_input = tf.random.normal([1, 3, 12000, 1])
test_output = generator([test_noise, test_input])
print("Generator output shape:", test_output.shape)

# Print model summary
generator.summary()

# Check intermediate layer shapes
for layer in generator.layers:
    if isinstance(layer, tf.keras.layers.Conv2D) or isinstance(layer, tf.keras.layers.Conv2DTranspose):
        print(f"Layer {layer.name} output shape: {layer.output_shape}")

discriminator = build_discriminator()

# Optimizers with gradient clipping
generator_optimizer = Adam(learning_rate=adam_lr, beta_1=adam_beta_1, clipvalue=1.0)
discriminator_optimizer = Adam(learning_rate=adam_lr, beta_1=adam_beta_1, clipvalue=1.0)

# Loss function
binary_cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def generator_loss(fake_output, denoised_waveforms, clean_waveforms):
    gan_loss = binary_cross_entropy(tf.ones_like(fake_output), fake_output)
    denoised_waveforms = tf.cast(denoised_waveforms, tf.float32)
    clean_waveforms = tf.cast(clean_waveforms, tf.float32)
    l1_loss = tf.reduce_mean(tf.abs(clean_waveforms - denoised_waveforms))
    total_loss = gan_loss + lambda_l1 * l1_loss
    tf.debugging.check_numerics(total_loss, "Generator loss is not finite")
    return gan_loss + lambda_l1 * l1_loss

def discriminator_loss(real_output, fake_output):
    real_loss = binary_cross_entropy(tf.ones_like(real_output) * 0.9, real_output)
    fake_loss = binary_cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    tf.debugging.check_numerics(total_loss, "Discriminator loss is not finite")
    return real_loss + fake_loss

# Add L1 loss for direct signal comparison
def l1_loss(y_true, y_pred):
    return tf.reduce_mean(tf.abs(y_true - y_pred))

#@tf.function
def train_step(noisy_waveforms, clean_waveforms):
    logger.info("Entering train_step")
    noisy_waveforms = tf.cast(noisy_waveforms, tf.float32)
    clean_waveforms = tf.cast(clean_waveforms, tf.float32)
    batch_size = tf.shape(noisy_waveforms)[0]
    noise = tf.random.normal([batch_size, latent_size])
    
    logger.info("Starting gradient tape")
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        logger.info("Generating waveforms")
        generated_waveforms = generator([noise, noisy_waveforms], training=True)
        
        '''
        '''
        tf.print("Noisy waveforms shape:", tf.shape(noisy_waveforms))
        tf.print("Generated waveforms shape:", tf.shape(generated_waveforms))
        tf.print("Clean waveforms shape:", tf.shape(clean_waveforms))
        '''
        '''
        tf.debugging.assert_equal(tf.shape(generated_waveforms), tf.shape(clean_waveforms),
                                  message="Shape mismatch between generated and clean waveforms")
        
        logger.info("Running discriminator on clean waveforms")
        real_output = discriminator(clean_waveforms, training=True)
        logger.info("Running discriminator on generated waveforms")
        fake_output = discriminator(generated_waveforms, training=True)
        
        logger.info("Calculating losses")
        gen_loss = generator_loss(fake_output, generated_waveforms, clean_waveforms)
        disc_loss = discriminator_loss(real_output, fake_output)
    
    logger.info("Calculating gradients")
    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
    
    if any(g is None for g in gradients_of_generator):
        logger.warning("Some generator gradients are None")
    if any(g is None for g in gradients_of_discriminator):
        logger.warning("Some discriminator gradients are None")
        
    logger.info("Applying gradients")
    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))
    
    logger.info("Exiting train_step")
    return gen_loss, disc_loss

def validation_step(X_val, y_val):
    X_val = tf.cast(X_val, tf.float32)
    y_val = tf.cast(y_val, tf.float32)
    noise = tf.random.normal([len(X_val), latent_size])
    generated_waveforms = generator([noise, X_val], training=False)
    
    real_output = discriminator(y_val, training=False)
    fake_output = discriminator(generated_waveforms, training=False)
    
    gen_loss = generator_loss(fake_output, generated_waveforms, y_val)
    disc_loss = discriminator_loss(real_output, fake_output)
    
    # Calculate PSNR and SSIM
    psnr = tf.reduce_mean(tf.image.psnr(y_val, generated_waveforms, max_val=1.0))
    ssim = tf.reduce_mean(tf.image.ssim(y_val, generated_waveforms, max_val=1.0))
    
    return disc_loss, gen_loss, psnr, ssim

train_dataset = tf.data.Dataset.from_tensor_slices((W_train, Y_train)).shuffle(1000).batch(batch_size).prefetch(tf.data.AUTOTUNE)
val_dataset = tf.data.Dataset.from_tensor_slices((W_val, Y_val)).batch(batch_size)
total_batches = tf.data.experimental.cardinality(train_dataset).numpy()
logger.info(f'Total number of batches: {total_batches}')

train_history = {'generator': [], 'discriminator': []}
val_history = {'generator': [], 'discriminator': [], 'psnr': [], 'ssim': []}

best_val_loss = float('inf')
best_epoch = 0

print(W.shape, Y.shape)


# Training loop
prev_gen_loss = None
prev_disc_loss = None
for epoch in range(nb_epochs):
    logger.info(f'Epoch {epoch + 1} of {nb_epochs}')
    
    # Training
    epoch_gen_loss = []
    epoch_disc_loss = []
    for i, (noisy_batch, clean_batch) in enumerate(train_dataset):
        
        logger.info(f'  Batch {i+1}')
        try:
            gen_loss, disc_loss = train_step(noisy_batch, clean_batch)
            if prev_gen_loss is not None and prev_disc_loss is not None:
                if tf.math.abs(gen_loss - prev_gen_loss) < 1e-6 and tf.math.abs(disc_loss - prev_disc_loss) < 1e-6:
                    logger.warning('Loss values are not changing significantly')
            prev_gen_loss = gen_loss
            prev_disc_loss = disc_loss
            epoch_gen_loss.append(gen_loss)
            epoch_disc_loss.append(disc_loss)
            logger.info(f'    Gen Loss: {gen_loss.numpy()}, Disc Loss: {disc_loss.numpy()}')
        except Exception as e:
            logger.error(f"Error in batch {i+1}: {str(e)}")
            import traceback
            logger.error(traceback.format_exc())
            break
    
    train_history['discriminator'].append(tf.reduce_mean(epoch_disc_loss))
    train_history['generator'].append(tf.reduce_mean(epoch_gen_loss))
    
    # Validation
    logger.info(f'Validation for epoch {epoch + 1}:')
    val_disc_loss = []
    val_gen_loss = []
    val_psnr = []
    val_ssim = []
    for noisy_batch, clean_batch in val_dataset:
        disc_loss, gen_loss, psnr, ssim = validation_step(noisy_batch, clean_batch)
        val_disc_loss.append(disc_loss)
        val_gen_loss.append(gen_loss)
        val_psnr.append(psnr)
        val_ssim.append(ssim)
    
    val_history['discriminator'].append(tf.reduce_mean(val_disc_loss))
    val_history['generator'].append(tf.reduce_mean(val_gen_loss))
    val_history['psnr'].append(tf.reduce_mean(val_psnr))
    val_history['ssim'].append(tf.reduce_mean(val_ssim))
    
    logger.info(f"Validation Discriminator Loss: {val_history['discriminator'][-1]}")
    logger.info(f"Validation Generator Loss: {val_history['generator'][-1]}")
    logger.info(f"Validation PSNR: {val_history['psnr'][-1]}")
    logger.info(f"Validation SSIM: {val_history['ssim'][-1]}")
    
    # Check if this is the best model so far
    current_val_loss = val_history['discriminator'][-1] + val_history['generator'][-1]
    if current_val_loss < best_val_loss:
        best_val_loss = current_val_loss
        best_epoch = epoch
        # Save the best model
        generator.save('/path/to/best_generator.h5')
        discriminator.save('/path/to/best_discriminator.h5')
    
    logger.info(f"Epoch {epoch+1}/{nb_epochs} completed. Current best epoch: {best_epoch+1}")

logger.info(f"Training completed. Best epoch was {best_epoch+1}")

# Final test on the best model
generator = tf.keras.models.load_model('/path/to/best_generator.h5')
discriminator = tf.keras.models.load_model('/path/to/best_discriminator.h5')

test_dataset = tf.data.Dataset.from_tensor_slices((W_test, Y_test)).batch(batch_size)
test_disc_loss = []
test_gen_loss = []
test_psnr = []
test_ssim = []

for noisy_batch, clean_batch in test_dataset:
    disc_loss, gen_loss, psnr, ssim = validation_step(noisy_batch, clean_batch)
    test_disc_loss.append(disc_loss)
    test_gen_loss.append(gen_loss)
    test_psnr.append(psnr)
    test_ssim.append(ssim)

logger.info(f"Final Test Discriminator Loss: {tf.reduce_mean(test_disc_loss)}")
logger.info(f"Final Test Generator Loss: {tf.reduce_mean(test_gen_loss)}")
logger.info(f"Final Test PSNR: {tf.reduce_mean(test_psnr)}")
logger.info(f"Final Test SSIM: {tf.reduce_mean(test_ssim)}")

# Save the training history for later analysis
np.save('/path/to/train_history.npy', train_history)
np.save('/path/to/val_history.npy', val_history)
'''

In [ ]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
import numpy as np
import logging
import matplotlib.pyplot as plt
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)
from tensorflow.keras.constraints import Constraint
import tensorflow.keras.losses as losses
from tensorflow_addons.layers import SpectralNormalization

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Hyperparameters
nb_epochs = 20
batch_size = 32
latent_size = 100
latent_dim = 100
adam_lr = 0.0002
adam_beta_1 = 0.5
lambda_l1 = 100  # Weight for L1 loss

# Loss functions
binary_cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=False)
'''
def generator_loss(fake_output, denoised_waveforms, clean_waveforms):
    fake_output = tf.cast(fake_output, tf.float32)
    gan_loss = binary_cross_entropy(tf.ones_like(fake_output), fake_output)
    denoised_waveforms = tf.cast(denoised_waveforms, tf.float32)
    clean_waveforms = tf.cast(clean_waveforms, tf.float32)
    l1_loss = tf.reduce_mean(tf.abs(clean_waveforms - denoised_waveforms))
    total_loss = gan_loss + lambda_l1 * l1_loss
    return total_loss, gan_loss, l1_loss
'''
def discriminator_loss(real_output, fake_output):
    real_loss = binary_cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = binary_cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss


# Define loss functions
bce_loss = losses.BinaryCrossentropy(from_logits=False)
mae_loss = losses.MeanAbsoluteError()

def generator_loss(validity, generated_waveforms, clean_waveforms):
    gan_loss = bce_loss(tf.ones_like(validity), validity)
    l1_loss = mae_loss(clean_waveforms, generated_waveforms)
    return gan_loss + lambda_l1 * l1_loss
'''
# Training step
@tf.function
def train_step(noisy_waveforms, clean_waveforms, noise):
    print("Entering train_step")
    noisy_waveforms = tf.cast(noisy_waveforms, tf.float32)
    clean_waveforms = tf.cast(clean_waveforms, tf.float32)
    
    # Generate noise for the generator input
    batch_size = tf.shape(noisy_waveforms)[0]
    #noise = tf.random.normal([batch_size, 100])  # Assuming latent_dim=100
    
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        # Generate denoised waveforms
        generated_waveforms = generator([noisy_waveforms, noise], training=True)
        
        # Discriminator predictions
        real_output = discriminator(clean_waveforms, training=True)
        fake_output = discriminator(generated_waveforms, training=True)
        
        # Calculate losses
        gen_total_loss, gen_gan_loss, gen_l1_loss = generator_loss(fake_output, generated_waveforms, clean_waveforms)
        disc_loss = discriminator_loss(real_output, fake_output)
    
    # Calculate gradients
    gen_gradients = gen_tape.gradient(gen_total_loss, generator.trainable_variables)
    disc_gradients = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
    
    # Clip gradients
    gen_gradients, _ = tf.clip_by_global_norm(gen_gradients, 1.0)
    disc_gradients, _ = tf.clip_by_global_norm(disc_gradients, 1.0)
    
    # Apply gradients
    generator_optimizer.apply_gradients(zip(gen_gradients, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(disc_gradients, discriminator.trainable_variables))
    
    return gen_total_loss, gen_gan_loss, gen_l1_loss, disc_loss
'''

@tf.function
def train_step(noisy_waveforms, clean_waveforms, noise):
    noisy_waveforms = tf.cast(noisy_waveforms, tf.float32)
    clean_waveforms = tf.cast(clean_waveforms, tf.float32)
    
    # Train discriminator
    with tf.GradientTape() as disc_tape:
        generated_waveforms = generator([noisy_waveforms, noise], training=True)
        real_output = discriminator(clean_waveforms, training=True)
        fake_output = discriminator(generated_waveforms, training=True)
        disc_loss = discriminator_loss(real_output, fake_output)
    
    disc_gradients = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
    discriminator_optimizer.apply_gradients(zip(disc_gradients, discriminator.trainable_variables))
    
    # Train generator (using combined model)
    with tf.GradientTape() as gen_tape:
        validity, generated_waveforms = combined_model([noisy_waveforms, noise], training=True)
        gen_loss = generator_loss(validity, generated_waveforms, clean_waveforms)
    
    gen_gradients = gen_tape.gradient(gen_loss, generator.trainable_variables)
    generator_optimizer.apply_gradients(zip(gen_gradients, generator.trainable_variables))
    
    return gen_loss, disc_loss


def cal_mse(y_true, y_pred):
    return tf.reduce_mean(tf.square(y_true - y_pred))

def cal_snr(y_true, y_pred):
    noise = y_true - y_pred
    return 10.0 * tf.math.log(tf.reduce_sum(tf.square(y_true)) / tf.reduce_sum(tf.square(noise))) / tf.math.log(10.0)

def validation_step(noisy_batch, clean_batch, noise):
    # Cast inputs to float32
    noisy_batch = tf.cast(noisy_batch, tf.float32)
    clean_batch = tf.cast(clean_batch, tf.float32)
    
    # Generate noise for the generator input
    batch_size = tf.shape(noisy_batch)[0]
    #noise = tf.random.normal([batch_size, 100])  # Assuming latent_dim=100
    
    generated_waveforms = generator([noisy_batch, noise], training=False)
    
    # Ensure generated_waveforms is float32
    generated_waveforms = tf.cast(generated_waveforms, tf.float32)
    
    real_output = discriminator(clean_batch, training=False)
    fake_output = discriminator(generated_waveforms, training=False)
    
    gen_total_loss = generator_loss(fake_output, generated_waveforms, clean_batch)
    disc_loss = discriminator_loss(real_output, fake_output)
    
    # Calculate MSE and SNR
    mse_value = cal_mse(clean_batch, generated_waveforms)
    snr_value = cal_snr(clean_batch, generated_waveforms)
    return gen_total_loss, disc_loss, mse_value, snr_value

# Build models (assuming you've defined build_generator and build_discriminator functions)
generator = build_generator()
discriminator = build_discriminator()

def build_combined_model(generator, discriminator):
    discriminator.trainable = False
    noise_input = tf.keras.layers.Input(shape=(latent_dim,))
    waveform_input = tf.keras.layers.Input(shape=(3, 12000, 1))
    generated_waveform = generator([waveform_input, noise_input])
    validity = discriminator(generated_waveform)
    combined = tf.keras.Model(inputs=[waveform_input, noise_input], outputs=[validity, generated_waveform])
    return combined

# Optimizers
lr_scheduler = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=adam_lr,
    decay_steps=1000,
    decay_rate=0.96
)
generator_optimizer = Adam(learning_rate=lr_scheduler, beta_1=adam_beta_1)
discriminator_optimizer = Adam(learning_rate=lr_scheduler, beta_1=adam_beta_1)

combined_model = build_combined_model(generator, discriminator)
combined_model.compile(loss=generator_loss, optimizer=generator_optimizer)


# Prepare datasets
train_dataset = tf.data.Dataset.from_tensor_slices((W_train, Y_train)).shuffle(1000).batch(batch_size)
val_dataset = tf.data.Dataset.from_tensor_slices((W_val, Y_val)).batch(batch_size)
test_dataset = tf.data.Dataset.from_tensor_slices((W_test, Y_test)).batch(batch_size)

# Training history
train_history = {'generator': [], 'discriminator': []}
val_history = {'generator': [], 'discriminator': [], 'mse': [], 'snr': []}

# Best model tracking
best_val_loss = float('inf')
best_epoch = 0

patience = 5
wait = 0
best_val_loss = float('inf')

# Training loop
for epoch in range(nb_epochs):
    print(f'Epoch {epoch + 1} of {nb_epochs}')
    
    # Training
    epoch_gen_loss = []
    epoch_disc_loss = []
    for i, (noisy_batch, clean_batch) in enumerate(train_dataset):
        print(f'  Batch {i+1}')
        try:
            noise = tf.random.normal([tf.shape(noisy_batch)[0], 100])
            gen_loss, disc_loss = train_step(noisy_batch, clean_batch, noise)
            
            epoch_gen_loss.append(gen_loss)
            epoch_disc_loss.append(disc_loss)
            print(f'Gen Total Loss: {gen_loss.numpy()}, Disc Loss: {disc_loss.numpy()}')
        except Exception as e:
            print(f'Error in batch {i+1}: {str(e)}')
            import traceback
            logger.error(traceback.format_exc())
            break
    
    train_history['generator'].append(tf.reduce_mean(epoch_gen_loss))
    train_history['discriminator'].append(tf.reduce_mean(epoch_disc_loss))
    
    # Validation
    print(f'Validation for epoch {epoch + 1}:')
    val_gen_loss = []
    val_disc_loss = []
    val_mse = []
    val_snr = []
    for noisy_batch, clean_batch in val_dataset:
        noise = tf.random.normal([tf.shape(noisy_batch)[0], 100])
        gen_loss, disc_loss, mse_value, snr_value = validation_step(noisy_batch, clean_batch, noise)
        val_gen_loss.append(gen_loss)
        val_disc_loss.append(disc_loss)
        val_mse.append(mse_value)
        val_snr.append(snr_value)
    
    print(f"Validation Generator Loss: {tf.reduce_mean(val_gen_loss)}")
    print(f"Validation Discriminator Loss: {tf.reduce_mean(val_disc_loss)}")
    print(f"Validation MSE: {tf.reduce_mean(val_mse)}")
    print(f"Validation SNR: {tf.reduce_mean(val_snr)}")
    
    val_history['generator'].append(tf.reduce_mean(val_gen_loss))
    val_history['discriminator'].append(tf.reduce_mean(val_disc_loss))
    val_history['mse'].append(tf.reduce_mean(val_mse))
    val_history['snr'].append(tf.reduce_mean(val_snr))
    
    # Check if this is the best model so far
    current_val_loss = val_history['generator'][-1] + val_history['discriminator'][-1]
    if current_val_loss < best_val_loss:
        best_val_loss = current_val_loss
        wait = 0
    else:
        wait += 1
        if wait >= patience:
            print(f"Early stopping triggered at epoch {epoch+1}")
            break
        best_epoch = epoch
        # Save the best model
        generator.save('/kaggle/working/best_generator.h5')
        discriminator.save('/kaggle/working/best_discriminator.h5')
        
    # Inside your training loop, after validation and before the next epoch
    if epoch % 1 == 0:  # You can adjust this to visualize less frequently if needed
        # Select a single noisy sample from your validation set
        noisy_sample = next(iter(val_dataset))[0][0]
        noisy_sample = tf.expand_dims(noisy_sample, 0)  # Add batch dimension

        # Generate noise
        noise = tf.random.normal([1, latent_dim])

        # Generate denoised sample
        generated_sample = generator([noisy_sample, noise], training=False)

        # Plot the results
        fig, axes = plt.subplots(1, 2, figsize=(12, 4))
        axes[0].imshow(tf.squeeze(noisy_sample).numpy(), aspect='auto', cmap='gray')
        axes[0].set_title('Noisy Input')
        axes[0].axis('off')

        axes[1].imshow(tf.squeeze(generated_sample).numpy(), aspect='auto', cmap='gray')
        axes[1].set_title(f'Denoised Output (Epoch {epoch + 1})')
        axes[1].axis('off')

        plt.tight_layout()
        plt.savefig(f'denoising_progress_epoch_{epoch + 1}.png')
        plt.close()

    print(f'Progress image saved.')
    print(f"Epoch {epoch+1}/{nb_epochs} completed. Current best epoch: {best_epoch+1}")
    
    
print(f"Training completed. Best epoch was {best_epoch+1}")

# Final test on the best model
custom_objects = {'SpectralNormalizationConstraint': SpectralNormalizationConstraint}

generator = tf.keras.models.load_model('/kaggle/working/best_generator.h5', custom_objects=custom_objects)
discriminator = tf.keras.models.load_model('/kaggle/working/best_discriminator.h5', custom_objects=custom_objects)

test_gen_loss = []
test_disc_loss = []
test_mse = []
test_snr = []

for noisy_batch, clean_batch in test_dataset:
    noise = tf.random.normal([tf.shape(noisy_batch)[0], 100])
    gen_loss, disc_loss, mse, snr = validation_step(noisy_batch, clean_batch, noise)
    test_gen_loss.append(gen_loss)
    test_disc_loss.append(disc_loss)
    test_mse.append(mse)
    test_snr.append(snr)

print(f"Final Test Generator Loss: {tf.reduce_mean(test_gen_loss)}")
print(f"Final Test Discriminator Loss: {tf.reduce_mean(test_disc_loss)}")
print(f"Final Test PSNR: {tf.reduce_mean(test_mse)}")
print(f"Final Test SSIM: {tf.reduce_mean(test_snr)}")

# Save the training history for later analysis
np.save('/kaggle/working/train_history.npy', train_history)
np.save('/kaggle/working/val_history.npy', val_history)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

# Create plotting lists
epochs = range(1, nb_epochs + 1)
train_gen_losses = [loss.numpy() for loss in train_history['generator']]
train_disc_losses = [loss.numpy() for loss in train_history['discriminator']]
val_gen_losses = [loss.numpy() for loss in val_history['generator']]
val_disc_losses = [loss.numpy() for loss in val_history['discriminator']]
val_mses = [mse.numpy() for mse in val_history['mse']]
val_snrs = [snr.numpy() for snr in val_history['snr']]

# Load best weights
generator.load_weights('/kaggle/working/best_generator.h5')
discriminator.load_weights('/kaggle/working/best_discriminator.h5')

# Function to generate and plot images
def generate_and_plot_images(model, test_input, test_output, num_examples=6, latent_dim=100):
    
    batch_size = test_input.shape[0]
    noise = tf.random.normal([batch_size, latent_dim])
    
    predictions = model([test_input, noise], training=False)
    
    fig = plt.figure(figsize=(18, 5*num_examples))
    
    for i in range(num_examples):
        # Plot noisy input
        ax = fig.add_subplot(num_examples, 3, i*3 + 1)
        plt.plot(test_input[i, :, 0, 0])
        plt.title('Noisy')
        plt.axis('off')
        
        # Plot generated output
        ax = fig.add_subplot(num_examples, 3, i*3 + 2)
        plt.plot(predictions[i, :, 0, 0])
        plt.title('Generated')
        plt.axis('off')
        
        # Plot ground truth
        ax = fig.add_subplot(num_examples, 3, i*3 + 3)
        plt.plot(test_output[i, :, 0, 0])
        plt.title('Ground Truth')
        plt.axis('off')
    
    plt.tight_layout()
    plt.savefig('/kaggle/working/generated_images.png')
    plt.show()
    
    return predictions
    
# Generate images
test_samples = next(iter(test_dataset))
predictions = generate_and_plot_images(generator, test_samples[0], test_samples[1])

# After generation
print("Output shape:", predictions.shape)
print("Output dtype:", predictions.dtype)
print("Output range:", tf.reduce_min(predictions).numpy(), tf.reduce_max(predictions).numpy())


# Final testing
test_gen_loss = []
test_disc_loss = []
test_mse = []
test_snr = []

for noisy_batch, clean_batch in test_dataset:
    batch_size = noisy_batch.shape[0]
    latent_dim=100
    noise = tf.random.normal([batch_size, latent_dim])
    generated_waveforms = generator([noisy_batch, noise], training=False)
    
    real_output = discriminator(clean_batch, training=False)
    fake_output = discriminator(generated_waveforms, training=False)
    
    gen_total_loss, gen_gan_loss, gen_l1_loss = generator_loss(fake_output, generated_waveforms, clean_batch)
    disc_loss = discriminator_loss(real_output, fake_output)
    
    gen_loss, disc_loss, mse_value, snr_value = validation_step(noisy_batch, clean_batch, noise)
    
    test_gen_loss.append(gen_total_loss)
    test_disc_loss.append(disc_loss)
    test_mse.append(mse_value)
    test_snr.append(snr_value)

# Check a few values
print("Sample input values:", test_samples[0][0, :5, 0, 0])
print("Sample output values:", predictions[0, :5, 0, 0])
print("Sample ground truth values:", test_samples[1][0, :5, 0, 0])    
    
# Print final test results
print(f"Final Test Generator Loss: {tf.reduce_mean(test_gen_loss)}")
print(f"Final Test Discriminator Loss: {tf.reduce_mean(test_disc_loss)}")
print(f"Final Test MSE: {tf.reduce_mean(test_mse)}")
print(f"Final Test SNR: {tf.reduce_mean(test_snr)}")

# Plot training and validation losses
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(epochs, train_gen_losses, label='Train Generator')
plt.plot(epochs, val_gen_losses, label='Val Generator')
plt.title('Generator Losses')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(epochs, train_disc_losses, label='Train Discriminator')
plt.plot(epochs, val_disc_losses, label='Val Discriminator')
plt.title('Discriminator Losses')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.savefig('/kaggle/working/loss_plots.png')
plt.show()

# Plot MSE and SNR
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(epochs, val_mses)
plt.title('Validation MSE')
plt.xlabel('Epoch')
plt.ylabel('MSE')

plt.subplot(1, 2, 2)
plt.plot(epochs, val_snrs)
plt.title('Validation SNR')
plt.xlabel('Epoch')
plt.ylabel('SNR')

plt.tight_layout()
plt.savefig('/kaggle/working/metrics_plots.png')
plt.show()

In [ ]:
test_samples[0]

In [ ]:
import pywt
import numpy as np

def wavelet_denoise(data, wavelet='db1', level=1):
    coeffs = pywt.wavedec(data, wavelet, mode='soft')
    threshold = np.sqrt(2 * np.log(len(data)))
    coeffs[1:] = (pywt.threshold(i, value=threshold, mode='soft') for i in coeffs[1:])
    return pywt.waverec(coeffs, wavelet, mode='soft')

# Example usage
denoised_waveforms = np.array([wavelet_denoise(w) for w in W_test])


In [ ]:
from skimage.restoration import estimate_sigma, denoise_nl_means
import numpy as np

def nlm_denoise(data, patch_size=5, patch_distance=3, h=0.8):
    sigma_est = np.mean(estimate_sigma(data))
    denoised = denoise_nl_means(data, h=h * sigma_est, fast_mode=True,
                                patch_size=patch_size, patch_distance=patch_distance)
    return denoised

# Example usage: Process only the first batch
batch_size = 10
denoised_waveforms_nlm = []

print(f"Starting denoising process for {len(W_test)} waveforms in batches of {batch_size}...")

# Process only the first batch
start_index = 0
end_index = batch_size
batch = W_test[start_index:end_index]

print(f"Processing batch 1 / {len(W_test) // batch_size + 1}")
denoised_batch = [nlm_denoise(w) for w in batch]
denoised_waveforms_nlm.extend(denoised_batch)
print("Finished batch 1")

denoised_waveforms_nlm = np.array(denoised_waveforms_nlm)

print("Denoising process for the first batch completed.")



In [ ]:
import numpy as np
from sklearn.metrics import mean_squared_error

# Function to calculate SNR
def calculate_snr(signal, noise):
    signal_power = np.mean(signal**2)
    noise_power = np.mean(noise**2)
    snr = 10 * np.log10(signal_power / noise_power)
    return snr

# Load your test data
X_test, y_test = W_test, Y_test

# Wavelet Denoising
denoised_waveforms_wavelet = np.array([wavelet_denoise(w) for w in X_test])
mse_wavelet = mean_squared_error(y_test.flatten(), denoised_waveforms_wavelet.flatten())
snr_wavelet = calculate_snr(y_test.flatten(), y_test.flatten() - denoised_waveforms_wavelet.flatten())

print(f"Wavelet Denoising - MSE: {mse_wavelet}, SNR: {snr_wavelet} dB")

# NLM Denoising
batch = 10
start_index = 0
end_index = batch_size
batch = W_test[start_index:end_index]

denoised_waveforms_nlm = np.array([nlm_denoise(w) for w in batch])
mse_nlm = mean_squared_error(y_test.flatten(), denoised_waveforms_nlm.flatten())
snr_nlm = calculate_snr(y_test.flatten(), y_test.flatten() - denoised_waveforms_nlm.flatten())

print(f"NLM Denoising - MSE: {mse_nlm}, SNR: {snr_nlm} dB")
